## Обучение Ранжирования

In [ ]:
!pip install --quiet -U  pyserini jsonlines scikit-surprise fastrank

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
DATA_PATH = './data'
!mkdir data
!cp /content/drive/MyDrive/DATASETS/MachineLearning/topicmodeling/lenta-ru-filtered.csv data
!cp /content/drive/MyDrive/DATASETS/MachineLearning/learningrank/MSLR-WEB10K.zip data/
!unzip data/MSLR-WEB10K.zip -d data/mslr

## Библиотеки

In [4]:
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
from matplotlib import gridspec

from tqdm.notebook import tqdm
import numpy as np
import  pandas as pd
import seaborn as sns
import torch
import scipy
import json
import re
import os
import jsonlines
import lightgbm as lgb

from surprise import Dataset, Reader, SVD
from fastrank import CDataset, TrainRequest
from sklearn.datasets import load_svmlight_file

from pyserini.search import SimpleSearcher

## Learninng to Rank

### Загружаем датасет

In [5]:
DATA_PATH = 'data/mslr/Fold1'
(X_train, y_train, qid_train) = load_svmlight_file(
    os.path.join(DATA_PATH, 'train.txt'), 
    dtype=np.float32, 
    zero_based=False,
    query_id=True)
(X_val, y_val, qid_val) = load_svmlight_file(
    os.path.join(DATA_PATH, 'vali.txt'), 
    dtype=np.float32, 
    zero_based=False,
    query_id=True)
(X_test, y_test, qid_test) = load_svmlight_file(
    os.path.join(DATA_PATH, 'test.txt'), 
    dtype=np.float32, 
    zero_based=False,
    query_id=True)

train_size = 50000 # set -1 to all data
X_train = X_train.toarray()[:train_size]
y_train = y_train[:train_size]
qid_train = qid_train[:train_size]

val_size = 10000 # set -1 to all data
X_val = X_val.toarray()[:val_size]
y_val = y_val[:val_size]
qid_val = qid_val[:val_size]

test_size = 1000 # set -1 to all data
X_test = X_test.toarray()[:test_size]
y_test = y_test[:test_size]
qid_test = qid_test[:test_size]


In [22]:
ids = np.random.RandomState(42).randint(0, len(X_train), size=5)
(X_train[ids][:, :6], y_train[ids], qid_train[ids])

(array([[2.      , 0.      , 1.      , 1.      , 2.      , 1.      ],
        [3.      , 0.      , 3.      , 2.      , 3.      , 1.      ],
        [2.      , 1.      , 2.      , 0.      , 2.      , 1.      ],
        [2.      , 0.      , 1.      , 0.      , 2.      , 1.      ],
        [6.      , 2.      , 1.      , 1.      , 6.      , 0.857143]],
       dtype=float32),
 array([0., 0., 2., 0., 1.]),
 array([2071,  151, 4966, 5971, 1486]))

 ### Описание модели

<img src="images/learningrank.png" alt="drawing" width="300"/>

### Определяем модель

In [6]:
train_request = TrainRequest.random_forest()
params = train_request.params
params.num_trees = 100
params.feature_sampling_rate = 0.5
params.instance_sampling_rate = 0.5

params.quiet = True
params.seed = 42

### Специальный формат данных

In [7]:
dataset_train = CDataset.from_numpy(X_train, y_train, qid_train)
dataset_test = CDataset.from_numpy(X_test, y_test, qid_test)

features = dataset_train.feature_names()
train_queries = sorted(dataset_train.queries())
test_queries = sorted(dataset_test.queries())

train = dataset_train.subsample_queries(train_queries)
test = dataset_test.subsample_queries(test_queries)

In [8]:
model = train.train_model(train_request)

$$DCG@K = \sum_{k=1}^{K}\frac{r_{true}\bigr(\pi^{-1}\bigr(k\bigr)\bigr)}{\log_2\bigr(k+1\bigr)}, \quad IDCG@K=\sum_{k=1}^{K}\frac{1}{\log_2\bigr(k+1\bigr)}$$

$$nDCG@K = \frac{DCG@K}{IDCG@K}$$

In [9]:
 test.evaluate(model, "NDCG@5")

{'103': 0.38701867902589504,
 '118': 0.6537915313679492,
 '13': 0.19727843198297884,
 '28': 0.6333513308383866,
 '43': 0.6354626356138486,
 '58': 0.2178022835137004,
 '73': 0.1391168867210682,
 '88': 0.4411217097171026}

## Рекомендательная система

In [10]:
data = Dataset.load_builtin('ml-100k')
trainset = data.build_full_trainset()
dataframe = pd.DataFrame(
    trainset.all_ratings(),
    columns=['uid', 'iid', 'rating'])
dataframe.sample(5, random_state=42)

Dataset ml-100k could not be found. Do you want to download it? [Y/n] y
Trying to download dataset from http://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /root/.surprise_data/ml-100k


,uid,iid,rating
75721,688,145,1.0
80184,742,446,3.0
19864,116,113,5.0
76699,696,587,3.0
92991,877,325,3.0


In [11]:
algo = SVD()
algo = algo.fit(trainset)

In [12]:
(algo.predict('10', '4').est, 
 algo.predict('10', '5').est, 
 algo.predict('10', '6').est)

(3.8507038796655597, 3.844281437272545, 4.177095033021916)

## Поисковая система

### Загружаем данные

In [13]:
data = pd.read_csv('data/lenta-ru-filtered.csv')

In [23]:
data

,text,tags,len,date
0,С 1 сентября на всей территории России вводитс...,Все,1654,31-08-1999
1,"По сведениям миссии ООН, передаваемым РИА ""Нов...",Все,1086,31-08-1999
2,15 представителей национал-большевистской парт...,Все,1219,31-08-1999
3,Намеченная на сегодняшний день церемония вступ...,Все,3094,31-08-1999
4,"На юге Киргизии, а именно в Баткенском и Чон-А...",Все,1354,31-08-1999
...,...,...,...,...
863280,Популярное место среди туристов в Мурманской о...,Россия,1231,11-09-2020
863281,Рейтинги от международного рейтингового агентс...,NaN,1425,11-09-2020
863282,Российские ученые нашли в Якутии новый подвид ...,События,1299,11-09-2020
863283,Для указания коронавируса как причины смерти ч...,Общество,2061,11-09-2020


### Описания поисковика

![image](images/search.png)

### Генерация документов

In [ ]:
COLLECTION_DIR = 'data/collection/lenta'
if not os.path.exists(COLLECTION_DIR):
    os.makedirs(COLLECTION_DIR)

collection = data.values

bath_size = 32768
for ind in tqdm(range(0, len(collection), bath_size)):
    files = [{'id': ind+i, 
              'contents': doc[0],
              'tag': doc[1],
              'date': doc[-1]}
             for i, doc in enumerate(collection[ind:ind+bath_size])]
    with jsonlines.open(os.path.join(COLLECTION_DIR, 
                                     '{}.jsonl'.format(ind)), 
                        mode='w') as writer:
        writer.write_all(files)

In [25]:
with open(os.path.join(COLLECTION_DIR, 
                       '{}.jsonl'.format(ind))) as f:
  print('\n'.join(f.read().splitlines()[-5:]))

{"id": 863280, "contents": "Популярное место среди туристов в Мурманской области завалили мусором и навозом. Об этом сообщает местное издание «Мурманский вестник». По данным портала, речь идет о Лавнинских водопадах в Кольском районе региона, ставших одним из излюбленных мест отдыха северян, где можно «искупаться и сделать красивые фотографии». Местные жители жалуются, что туристы не убирают за собой мусор и пластиковые бутылки, а недавно там появились огромные лужи навоза. «На всем пути к Лавне \"аромат\" такой, аж оторопь берет... Особое негодование вызывают болотного вида лужи навоза, смердящего и отравляющего воду. А ведь лет пять назад мы набирали чайник прямо из водопада, не боялись отравиться нечистотами», — рассказала жительница района Лиза Титова. Откуда взялся навоз возле водопадов, неизвестно, но люди предполагают, что во всем могут быть виноваты фермеры, сливающие отходы на территорию курорта. В июне сообщалось, что жители английского города Бакстон подговорили фермеров заб

### Построение индекса

In [26]:
INDEX_DIR = 'data/index/lenta'
if not os.path.exists(INDEX_DIR):
    os.makedirs(INDEX_DIR)

In [ ]:
!python -m pyserini.index -collection JsonCollection -generator DefaultLuceneDocumentGenerator \
 -threads 16 -input {COLLECTION_DIR} \
 -index {INDEX_DIR} -storePositions -storeDocvectors -storeRaw

### Используем поисковый индекс

#### BM-25

Пусть дан запрос $Q$, содержащий слова $q_1, ..., q_n$, тогда функция BM25 даёт следующую оценку релевантности документа $D$ запросу $Q$:
$$\text{score}(D,Q) = \sum_{i=1}^{n} \text{IDF}(q_i) \cdot \frac{tf(q_i, D) \cdot (k_1 + 1)}{tf(q_i, D) + k_1 \cdot (1 - b + b \cdot \frac{|D|}{\text{avgdl}})},$$
где $tf(q_i, D)$ есть частота слова $q_i$ в документе $D$, $|D|$ есть длина документа (количество слов в нём), а $vgdl$ — средняя длина документа в коллекции. $k_1$ и $b$ — свободные коэффициенты.

In [28]:
searcher = SimpleSearcher(INDEX_DIR)
searcher.set_bm25()
hits = searcher.search('Поисковик', k=1)

for i in range(len(hits)):
    doc = searcher.doc(hits[i].docid)
    print(doc.raw())

{
  "id" : 331463,
  "contents" : "Группа хакеров пообещала возродить Usenet-поисковик Newzbin. Соответствующее письмо было разослано зарегистрированным пользователям сервиса, пишет TorrentFreak. В письме группа хакеров, называющая себя \"Team R Dogs\", сообщила, что ей удалось получить большую часть исходного кода и баз данных ресурса. Ранее бывший редактор поисковика под ником DeepSharer сообщил в своем блоге, что, по слухам, исходный код действительно был выкраден. С помощью украденных файлов \"Team R Dogs\" надеется запустить новый поисковик под именем Newzbin 2. При этом хакер под псевдонимом Mr. White заявил: \"Мы слишком сильно любили [этот ресурс], чтобы позволить ему умереть\". Он также сообщил, что поисковик заработает в ближайшее время. В марте 2010 года британский суд постановил, что поисковик Newzbin нарушает авторские права. Основанием для принятия такого решения стала жалоба, поданная Ассоциацией кинопроизводителей (MPA). Эта организация защищает права крупнейших киносту

#### QLD

Идея: латентное размещение Дирихле (отсылка к тематическим моделям)

In [29]:
searcher = SimpleSearcher(INDEX_DIR)
searcher.set_qld()
hits = searcher.search('Поисковик', k=1)

for i in range(len(hits)):
    doc = searcher.doc(hits[i].docid)
    print(doc.raw())

{
  "id" : 331463,
  "contents" : "Группа хакеров пообещала возродить Usenet-поисковик Newzbin. Соответствующее письмо было разослано зарегистрированным пользователям сервиса, пишет TorrentFreak. В письме группа хакеров, называющая себя \"Team R Dogs\", сообщила, что ей удалось получить большую часть исходного кода и баз данных ресурса. Ранее бывший редактор поисковика под ником DeepSharer сообщил в своем блоге, что, по слухам, исходный код действительно был выкраден. С помощью украденных файлов \"Team R Dogs\" надеется запустить новый поисковик под именем Newzbin 2. При этом хакер под псевдонимом Mr. White заявил: \"Мы слишком сильно любили [этот ресурс], чтобы позволить ему умереть\". Он также сообщил, что поисковик заработает в ближайшее время. В марте 2010 года британский суд постановил, что поисковик Newzbin нарушает авторские права. Основанием для принятия такого решения стала жалоба, поданная Ассоциацией кинопроизводителей (MPA). Эта организация защищает права крупнейших киносту

#### RM3

Пусть дан запрос $Q$, содержащий слова $q_1, ..., q_n$, тогда функция BM25 даёт следующую оценку релевантности токена $t$ запросу $Q$
$$score\bigr(t, Q\bigr) = \alpha \cdot \text{softmax}\left(\sum_{d\in PRD}\left[\frac{tf\bigr(t, d\bigr)}{|d|}\cdot\prod_{i=1}^{k}\frac{tf\bigr(q_i, d\bigr)+\mu p\bigr(q_i\bigr)}{|d|+\mu}\right]\right) + \left(1-\alpha\right)\cdot\left(\frac{tf\bigr(t, Q\bigr)}{|Q|}\right),$$
где $tf(t, d)$ есть частота слова $t$ в документе $d$, $|d|$ есть длина документа (количество слов в нём), а $\alpha$ и $\mu$ — свободные коэффициенты.

In [30]:
searcher = SimpleSearcher(INDEX_DIR)
searcher.set_rm3()
hits = searcher.search('Поисковик', k=1)

for i in range(len(hits)):
    doc = searcher.doc(hits[i].docid)
    print(doc.raw())

{
  "id" : 338651,
  "contents" : "Китайский поисковик Baidu разрабатывает собственную мобильную ОС с открытым исходным кодом. Об этом пишет InformationWeek со ссылкой на публикации в китайских газетах. C помощью новинки Baidu рассчитывает укрепить свои позиции на рынке мобильного поиска в стране. Также она сможет составить конкуренцию мобильной ОС Google Android, которая является одной из самых популярных мобильных платформ в мире. Сообщается, что разработкой ОС занимаются бывшие сотрудники Google, которые покинули китайское представительство американской компании после того, как та прекратила осуществлять цензуру поисковых запросов в Китае. Baidu является самой популярной поисковой системой в Китае. Сообщается, что ей принадлежит более 70 процентов рынка поисковых запросов в стране. При этом по количеству мобильных поисковых запросов Baidu находится приблизительно на одном уровне с Google - на эти поисковики приходится примерно по четверти от общего числа мобильного поиска. В собстве